<a href="https://colab.research.google.com/github/dia2teen/first_project/blob/master/!2_7_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pytorch_lightning
!pip install -q monai

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.cuda.get_device_name(0)


'Tesla T4'

In [ ]:
import pytorch_lightning as pl
import monai
import h5py

In [ ]:
%%capture
!if [ ! -e seq.h5 ]; then wget https://github.com/japan-medical-ai/medical-ai-course-materials/releases/download/v0.1/seq.h5; fi

In [ ]:
# データの読み込み
with h5py.File('seq.h5', 'r') as hf:
    x_train, t_train = hf['train_in'][()], hf['train_out'][()]
    x_val, t_val = hf['valid_in'][()], hf['valid_out'][()]
    x_test, t_test = hf['test_in'][()], hf['test_out'][()]
    label = hf['target_labels'][()]

In [ ]:
class SequenceDataset(torch.utils.data.Dataset):

    def __init__(self, x, t):
        self.x = x
        self.t = t[:, :, 1]  # 今回は 2 つ目の実験結果のカバレッジ値のみ使用

    def __getitem__(self, idx):
        x = torch.tensor(self.x[idx], dtype=torch.float32).permute(1, 0)
        t = torch.tensor(self.t[idx], dtype=torch.float32)
        return x, t

    def __len__(self):
        return len(self.x)

In [ ]:
# シードを固定
pl.seed_everything(0)

# データセットの取得
train = SequenceDataset(x_train, t_train)
val = SequenceDataset(x_val, t_val)
test = SequenceDataset(x_test, t_test)

# バッチサイズの定義
batch_size = 64

# Data Loader を定義
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

Global seed set to 0


In [ ]:
from monai.networks.blocks import Convolution


In [ ]:
!pip install -q optuna
import optuna
optuna.__version__

'2.3.0'

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.metrics import r2_score

In [ ]:
class Net2(pl.LightningModule):

    def __init__(self,step_size=800, gamma=0.3):
        super().__init__()
        self.step_size = step_size
        self.gamma = gamma

        self.squeeze_params = [
            {'in_channels': 4,  'out_channels': 28,  'kernel_size': 21, 'strides': 2},
            {'in_channels': 28, 'out_channels': 52,  'kernel_size': 7,  'strides': 4},
            {'in_channels': 52, 'out_channels': 76,  'kernel_size': 7,  'strides': 4},
            {'in_channels': 76, 'out_channels': 100, 'kernel_size': 7,  'strides': 4},
        ]

        self.dilated_conv_params = [
            {'in_channels': 100, 'out_channels': 24, 'kernel_size': 3, 'dilation': 1},
            {'in_channels': 124, 'out_channels': 24, 'kernel_size': 3, 'dilation': 2},
            {'in_channels': 148, 'out_channels': 24, 'kernel_size': 3, 'dilation': 4},
            {'in_channels': 172, 'out_channels': 24, 'kernel_size': 3, 'dilation': 8},
            {'in_channels': 196, 'out_channels': 24, 'kernel_size': 3, 'dilation': 16},
            {'in_channels': 220, 'out_channels': 24, 'kernel_size': 3, 'dilation': 32},
        ]

        # Squeeze Block
        self.squeeze = nn.Sequential()
        for (i, p) in enumerate(self.squeeze_params):
            layer = Convolution(
                      dimensions=1,
                      in_channels=p['in_channels'], out_channels=p['out_channels'],
                      kernel_size=p['kernel_size'], strides=p['strides'],
                      norm='batch', act='ReLu')
            self.squeeze.add_module(f'conv{i+1}', layer)

        # Dilated Block
        self.dilated_conv = nn.Sequential()
        for (i, p) in enumerate(self.dilated_conv_params):
            layer = Convolution(
                      dimensions=1,
                      in_channels=p['in_channels'], out_channels=p['out_channels'],
                      kernel_size=p['kernel_size'], dilation=p['dilation'],
                      norm='batch', act='ReLU')
            self.dilated_conv.add_module(f'conv_{i+1}', layer)

        # Convolution
        self.conv = nn.Conv1d(in_channels=244, out_channels=1, kernel_size=1)

    def forward(self, x):
        # Squeeze Block
        h = self.squeeze(x)
        # Dilated Block
        hs = [h]
        for layer in self.dilated_conv:
            h = torch.cat(hs, dim=1)
            h = layer(h)
            hs.append(h)
        # Convolution
        h = torch.cat(hs, dim=1)
        h = self.conv(h)
        h = h.squeeze()
        return h

    def r2_score(self, y, t):
        y = torch.exp(y)
        r_n = ((t - y)**2).sum(dim=0)
        r_d = ((t - t.mean(dim=0))**2).sum(dim=0)
        r = 1 - r_n / r_d
        return r.mean()

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.poisson_nll_loss(y, t)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_r2', self.r2_score(y, t), on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.poisson_nll_loss(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_r2', self.r2_score(y, t), on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.poisson_nll_loss(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_r2', self.r2_score(y, t), on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        # scheduler を追加
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = self.step_size, gamma = self.gamma)
        return [optimizer], [scheduler] #2つを返すようにする


In [ ]:
def objective(trial): 

  # ハイパーパラメータの候補
  step_size = trial.suggest_int('step_size', 100, 800)
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)

  # ネットワークの訓練（早期終了あり）
  pl.seed_everything(0)
  net2 = Net2(step_size, gamma)
  trainer2 = pl.Trainer(max_epochs=10, gpus=1, deterministic=True, callbacks=[EarlyStopping(monitor='val_r2')])
  trainer2.fit(net2, train_loader, val_loader)
  result = trainer2.callback_metrics['val_r2'] 

  # 検証データの accuracy で評価
  return result

In [ ]:
# study オブジェクトを作成し、最適化を実行
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=10)

In [ ]:
study.best_params

In [ ]:
study.best_value